# Finetune YOLOv8n-pose on COCO2017 (person only)
This notebook downloads COCO2017, filters out only human data, converts to YOLOv8 pose format, and prepares for training.

## Step 1: Download COCO2017 and Annotations

In [ ]:
import os
import json
import shutil
import argparse
from tqdm import tqdm
from PIL import Image

def coco_keypoints_to_yolo(annotation, img_w, img_h):
    x, y, w, h = annotation['bbox']
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w /= img_w
    h /= img_h

    keypoints = annotation['keypoints']
    kps_norm = []
    for i in range(17):
        kp_x = keypoints[i * 3] / img_w
        kp_y = keypoints[i * 3 + 1] / img_h
        v = keypoints[i * 3 + 2]
        kps_norm.extend([kp_x, kp_y, v])

    return [0, x_center, y_center, w, h] + kps_norm

def convert_coco_to_yolo_pose(image_dir, json_path, output_dir, label_dir):
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(label_dir, exist_ok=True)

    with open(json_path, 'r') as f:
        data = json.load(f)

    images = {img['id']: img for img in data['images']}

    annotations_by_image = {}
    for ann in data['annotations']:
        if ann['category_id'] != 1:
            continue
        if ann['image_id'] not in annotations_by_image:
            annotations_by_image[ann['image_id']] = []
        annotations_by_image[ann['image_id']].append(ann)

    for img_id, anns in tqdm(annotations_by_image.items()):
        img_info = images[img_id]
        file_name = img_info['file_name']
        src_path = os.path.join(image_dir, file_name)
        dst_path = os.path.join(output_dir, file_name)

        if not os.path.exists(src_path):
            continue

        shutil.copy(src_path, dst_path)
        img_w = img_info['width']
        img_h = img_info['height']

        label_lines = []
        for ann in anns:
            if ann['num_keypoints'] < 1:
                continue
            yolo_line = coco_keypoints_to_yolo(ann, img_w, img_h)
            label_lines.append(" ".join([str(round(x, 6)) if i > 0 else str(int(x)) for i, x in enumerate(yolo_line)]))

        label_file = os.path.join(label_dir, file_name.replace('.jpg', '.txt').replace('.png', '.txt'))
        with open(label_file, 'w') as f:
            f.write("\n".join(label_lines))

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--image_dir', required=True, help='Path to image folder')
    parser.add_argument('--json_path', required=True, help='Path to COCO keypoint json')
    parser.add_argument('--output_dir', required=True, help='Output image folder')
    parser.add_argument('--label_dir', required=True, help='Output label folder')
    args = parser.parse_args()

    convert_coco_to_yolo_pose(args.image_dir, args.json_path, args.output_dir, args.label_dir)

## Step 2: Convert COCO annotations (person only) to YOLOv8 pose format